# Libraries

In [1]:
# Libraries
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Input, Activation, Dropout, BatchNormalization, Conv2DTranspose, Concatenate 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.backend import clear_session
from random import shuffle

# U-net construction

In [2]:
def conv_block(input_layer, n_base, batchnorm = False):
    output_layer = Conv2D(filters= n_base, kernel_size=(3,3), strides=(1,1), padding='same')(input_layer)
    if batchnorm :
        output_layer = BatchNormalization()(output_layer)
    output_layer = Activation('relu')(output_layer)
    output_layer = Conv2D(filters=n_base, kernel_size=(3,3), strides=(1,1), padding='same')(output_layer)
    if batchnorm :
        output_layer = BatchNormalization()(output_layer)
    output_layer = Activation('relu')(output_layer)
    return output_layer

In [3]:
def encoder_block(input_layer,n_base , batchnorm = False, dropout = False):
    out = conv_block(input_layer, n_base, batchnorm = batchnorm)
    out2 = MaxPooling2D(pool_size=(2,2))(out)
    if dropout:
        out2 = Dropout(0.2)(out2)
    return out, out2

In [9]:
def decoder_block(input_layer, layer2conc, n_base, batchnorm = False, dropout = False):
    print(str(n_base), input_layer.shape)
    output_layer = Conv2DTranspose(filters = n_base,  kernel_size=(3,3), strides=(2, 2), padding="same")(input_layer)
    print(str(n_base), output_layer.shape)
    output_layer = Concatenate()([output_layer, layer2conc])
    if dropout:
        output_layer = Dropout(0.2)(output_layer)
    output_layer = conv_block(output_layer, n_base, batchnorm = batchnorm)
    return output_layer

In [10]:
def get_unet(img_ch, img_width, img_height, n_base, dropout = False, batchnormal = False, binary = True, class_num = 2):
    input_layer = Input(shape=(img_width, img_height, img_ch))
    
    #Encoder
    e1, em1 = encoder_block(input_layer, n_base, batchnorm = batchnormal, dropout = dropout)
    e2, em2 = encoder_block(em1, n_base*2, batchnorm = batchnormal, dropout = dropout)
    e3, em3 = encoder_block(em2, n_base*4, batchnorm = batchnormal, dropout = dropout)
    e4, em4 = encoder_block(em3, n_base*8, batchnorm = batchnormal, dropout = dropout)

    #Bottleneck 
    bottleneck = conv_block(em4, n_base*16, batchnorm = batchnormal)

    #Decoder
    d_block1 = decoder_block(bottleneck, e4, n_base*8, batchnorm = batchnormal, dropout = dropout)
    d_block2 = decoder_block(d_block1, e3, n_base*4, batchnorm = batchnormal, dropout = dropout)
    d_block3 = decoder_block(d_block2, e2, n_base*2, batchnorm = batchnormal, dropout = dropout)
    d_block4 = decoder_block(d_block3, e1, n_base, batchnorm = batchnormal, dropout = dropout)
    
    #Output
    if binary:
        out = Conv2D(filters=1, kernel_size=(3,3), strides=(1,1), padding='same', activation = 'sigmoid')(d_block4)
    else:
        out = Conv2D(filters=class_num, kernel_size=(3,3), strides=(1,1), padding='same', activation = 'sigmoid')(d_block4)
        
    clf = Model(inputs=input_layer, outputs=out)
    clf.summary()
    
    return clf   

In [11]:
clf = get_unet(1, 256, 256, 16, dropout = True, batchnormal = True)

128 (None, 16, 16, 256)
128 (None, 32, 32, 128)
64 (None, 32, 32, 128)
64 (None, 64, 64, 64)
32 (None, 64, 64, 64)
32 (None, 128, 128, 32)
16 (None, 128, 128, 32)
16 (None, 256, 256, 16)
Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 256, 256, 16) 160         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_54 (BatchNo (None, 256, 256, 16) 64          conv2d_57[0][0]                  
__________________________________________________________________________________________________

In [ ]:
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 0.0001) / (K.sum(y_true_f) + K.sum(y_pred_f) + 0.0001)

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

# Data loading

In [ ]:
import os
import numpy as np
from random import shuffle
from skimage.io import imread
from skimage.transform import resize

In [ ]:
def load_data(data_path, img_list, mask_list, state):
    #Checking that img_list and mask_list are in the same order
    mask_list2 = [mask.replace('_Tumor.png', '.png') for mask in mask_list]
    if mask_list2 == img_list:
        del mask_list2
        pass
    else :
        print('Images and masks are not in the same order')
    
    #Initialising the final arrays
    img_array = np.zeros((len(img_list), img_h, img_w), dtype = np.float32)
    mask_array = np.zeros((len(mask_list), img_h, img_w), dtype = np.float32)
    
    ind = 0
    for img_label, mask_label in zip(img_list, mask_list):
        #Loading image
        img = imread(data_path + 'Image/' + img_label, as_gray = True)
        img = resize(img, (img_h, img_w), anti_aliasing = True).astype('float32')
        img_array[ind] = (np.array(img)-np.min(np.array(img)))/(np.max(np.array(img))-np.min(np.array(img)))
        #Loading mask
        img = imread(data_path + 'Mask/' + mask_label, as_gray = True)
        img = resize(img, (img_h, img_w), anti_aliasing = True).astype('float32')
        mask_array[ind] = (np.array(img)-np.min(np.array(img)))/(np.max(np.array(img))-np.min(np.array(img)))
        #Update counter
        ind = ind + 1
        print(state + ': ' + str(ind) + '/' + str(len(img_list)))
    img_array = np.expand_dims(img_array, axis =3)
    mask_array = np.expand_dims(mask_array, axis =3)
    return img_array, mask_array

In [ ]:
img_h, img_w = 240, 240
path = '/DL_course_data/Lab3/MRI/'
img_list = os.listdir(path + 'Image/')
shuffle(img_list)
mask_list = [file.replace('.png', '_Tumor.png') for file in img_list]

# K fold

In [ ]:
def K_fold(x_list, y_list, path, K, current_index):
    n = len(x_list)
    x_test, y_test = load_data(path, 
                               x_list[int(current_index*n/K):int((current_index+1)*n/K)], 
                               y_list[int(current_index*n/K):int((current_index+1)*n/K)],
                               'Test')
    x_train, y_train = load_data(path, 
                                 x_list[:int(current_index*n/K)] + x_list[int((current_index+1)*n/K):], 
                                 y_list[:int(current_index*n/K)] + y_list[int((current_index+1)*n/K):],
                                 'Train')
    return x_train, x_test, y_train, y_test

# Task 1

In [ ]:
img_ch, img_h, img_w = 1, 240, 240
n_base = 8
batchsize = 8
LR = 1e-4
n_epochs = 100
K1 = 3
for fold in range(K1):
    #Creating the fold
    x_train, x_test, y_train, y_test = K_fold(img_list, mask_list, path, K1, fold)
    
    #Training the network
    clf = get_unet(img_ch, img_w, img_h, n_base, dropout = True, batchnormal = True)
    clf.compile(loss=[dice_coef_loss], optimizer = Adam(lr = LR), metrics=[dice_coef, Precision(), Recall()]) 
    clf_hist = clf.fit(x_train, y_train, epochs = n_epochs, batch_size = batchsize, validation_data=(x_test, y_test))
    
    #Free memory
    del x_train
    del y_train
    del x_test
    del y_test
    
    #Saving results
    #Loss
    plt.figure(figsize=(4, 4))
    plt.title("Learning curve")
    plt.plot(clf_hist.history["loss"], label="loss")
    plt.plot(clf_hist.history["val_loss"], label="val_loss")
    xmin = np.argmin(clf_hist.history["val_loss"])
    ymin = np.min(clf_hist.history["val_loss"])
    plt.plot( xmin, ymin, marker="x", color="r", label="best model")
    plt.annotate('(' + str(xmin) + ', '+ str(round(ymin, 2)) + ')', xy = (xmin, ymin - 0.01),
                 horizontalalignment = "center", verticalalignment = "top", color = "red")
    plt.xlabel("Epochs")
    plt.ylabel("Loss Value")
    plt.legend();
    plt.savefig('1/loss_fold-'+ str(fold+1) +'.png', dpi = 200)

    #Accuracy
    plt.figure(figsize=(4, 4))
    plt.title("Accuracy")
    plt.plot(clf_hist.history["dice_coef"], label="accuracy")
    plt.plot(clf_hist.history["val_dice_coef"], label="val_accuracy")
    xmax = np.argmax(clf_hist.history["val_dice_coef"])
    ymax = np.max(clf_hist.history["val_dice_coef"])
    plt.plot( xmax, ymax, marker="x", color="r", label="best model")
    plt.annotate('(' + str(xmax) + ', '+ str(round(ymax,2)) + ')', xy = (xmax, ymax + 0.01),
                 horizontalalignment = "center", verticalalignment = "bottom", color = "red")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy Value")
    plt.legend();
    plt.savefig('1/accuracy_fold-' + str(fold+1)+ '.png', dpi = 200)
    
    #Precision
    plt.figure(figsize=(4, 4))
    plt.title("Precision")
    plt.plot(clf_hist.history['precision'], label="Precision")
    plt.plot(clf_hist.history["val_precision"], label="val_precision")
    xmax = np.argmax(clf_hist.history["val_precision"])
    ymax = np.max(clf_hist.history["val_precision"])
    plt.plot( xmax, ymax, marker="x", color="r", label="best model")
    plt.annotate('(' + str(xmax) + ', '+ str(round(ymax,2)) + ')', xy = (xmax, ymax + 0.01),
                 horizontalalignment = "center", verticalalignment = "bottom", color = "red")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy Value")
    plt.legend();
    plt.savefig('1/precision_fold-' + str(fold+1)+ '.png', dpi = 200)
    
    # Recall
    plt.figure(figsize=(4, 4))
    plt.title("Recall")
    plt.plot(clf_hist.history['recall'], label="Recall")
    plt.plot(clf_hist.history['val_recall'], label="val_Recall")
    xmax = np.argmax(clf_hist.history['val_recall'])
    ymax = np.max(clf_hist.history['val_recall'])
    plt.plot( xmax, ymax, marker="x", color="r", label="best model")
    plt.annotate('(' + str(xmax) + ', '+ str(round(ymax,2)) + ')', xy = (xmax, ymax + 0.01),
                 horizontalalignment = "center", verticalalignment = "bottom", color = "red")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy Value")
    plt.legend();
    plt.savefig('1/recall_fold-' + str(fold+1)+ '.png', dpi = 200)
    
    del clf
    clear_session()